### 2018/2019 - Task List 10

1. Implement Naive Bayes classifier with pyro
    - create apropriate parameters (mean and std for a and b, sigma - noise)
    - provide optimization procedure
    - check appropriateness of implemented method with selected dataset


# Required imports

In [7]:
%matplotlib inline
import pyro
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.optim as optim
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive

In [8]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)

## Solutions

In [9]:
def read_dataset():    
    data_seeds = pd.read_csv('./data_sets/seeds.data', sep="\t", header=None)
    data_seeds.columns = ["Area A", "Perimeter P", "Compactness C = 4*pi*A/P^2", "Length of kernel",
                      "Width of kernel", "Asymmetry coefficient", "Length of kernel groove", "decision"]
    return data_seeds.iloc[:, :-1].get_values(), data_seeds.iloc[:, -1:].get_values()

# assuming normal distributuion of continuous attributes, we can use pdf instead of discretization
# def probability_density_function(mean, std, value):
#     # shouldn't happen
#     if std == 0:
#         std = 0.001
# #         return .001

#     return 1 / (std * np.sqrt(2 * np.pi)) * np.exp(- (value - mean) ** 2 / (2 * std ** 2))

In [10]:
def pdf_normal(value, mean, std):
    # shouldn't happen
    if std == 0:
        return .001

    return 1 / (std * np.sqrt(2 * np.pi)) * np.exp(- (value - mean) ** 2 / (2 * std ** 2))

def model(data_x, data_y):    
#     abc = pyro.sample("abc", dist.Beta(1, 1))
    apriori = pyro.sample("apriori", dist.Normal(torch.zeros(3, 2)))

    # ask
    # notation like this was on blackboard - what it suppose to mean?
    apriori = dist.Normal(torch.zeros(3, 2)) # dim: attr_num x decisions_num
    
    # ask
    # is this declaration ok?
    # and how to get to those indexes in plate?
    # dim: attr_num x decisions_num
    apriori_mean = torch.zeros(attr_num, decisions_num[-1])
    apriori_std = torch.zeros(attr_num, decisions_num[-1])
    
    # go throught decisions - unique data_y
    with pyro.plate("decisions", size=decisions_num):
        # sample prob of class
        # why wy sample class, while we are having it in plate?
        # dim: 1x2 -> 1 x decisions_num
        decision_prob = pyro.sample("decision", dist.Categorical(decisions_wages))
        
        with pyro.plate("attributes", size=attr_num) as value:
            # sampling from whole apriori?
            a = pyro.sample("a", dist.Normal(,[apriori])) # dim: 3x2 -> attr_num x decisions_num
            # ask
            # how to get the value - just x_data?
            pdf = pdf_normal(x_data, mean, std) # wym: n x 3 -> objects x attr_num
            
            pyro.sample("obs", dist.Categorical(pdf), obs=y_data)

def guide(data_x, data_y):
    # todo
    
    # would it work?
    pyro.param("", mean, tensor.zeros(attr_num, decisions_num[-1]))




def train(data_x, data_y):
    # some dimensions
    count = len(data_x)
    decisions_num = np.unique(data_y)
    attr_num = len(data_x[0])
    # todo change - modify it later in training
    decisions_wages = torch.tensor([1/decisions_num[-1]] * decisions_num[-1])
    
    pyro.clear_param_store()
    num_iterations = 3000
#     model = create_probabilistic_model()
    optim = pyro.optim.Adam({"lr": 0.005})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=count)
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(data_x, data_y)
        t.set_postfix(loss=loss)
    return (model, svi)

# def get_marginal(traces, sites):
#     empirical_marginal=EmpiricalMarginal(traces, sites)
#     return empirical_marginal._get_samples_and_weights()[0].detach().cpu().numpy()

# def wrapped_model(x_data, y_data):
#     model_result=probabilistic_model(x_data, y_data)
#     pyro.sample("prediction", pyro.distributions.Delta(model_result))

SyntaxError: invalid syntax (<ipython-input-10-80557f6e5c0f>, line 32)

In [11]:
data_x, data_y = read_dataset()

count = len(data_x)
decisions_num = np.unique(data_y)
attr_num = len(data_x[0])


# print(decisions_num)
# train(data_x, data_y)

# check parameters
# print("Check parameters:")
# for name, value in pyro.get_param_store().items():
#     print(name, pyro.param(name))

In [12]:


# apriori = pyro.sample("apriori", dist.Beta(1, 1))
# abc = pyro.sample("abs", dist.Normal(3, 2))

# with pyro.plate("map", size=2):
#     # c=pyro.sample("", Categorical(abc)) # wym: 1x2
#     a=pyro.sample("", dist.Normal([apriori])) # wym: 3x2
#     pdf = PDF(x_data) # wym: n x 3

#     pdf_c = abc * pdf # wym: n x 2

#     tst = pyro.sample(dist.Categorical(pdf_c)) # nx2
tst, tst_2 = read_dataset()
# tmp = pyro.sample("tmp", dist.Beta(1, 1))
# torch.tensor([ 0.25, 0.25, 0.25, 0.25 ])
# decisions_num = 5
# wages = torch.ones(1, decisions_num[-1])*[1/decisions_num] #torch.tensor([1/decisions_num])*3
wages = torch.tensor([1/decisions_num[-1]] * decisions_num[-1])
tmp = pyro.sample("tmp", dist.Categorical(wages))
apriori = dist.Normal(0.3, 0.1, torch.zeros(3, 2))

pyro.sample('tst', dist.Normal(torch.tensor.zeros([0]), torch.tensor(1)))
# print(1/decisions_num[-1])
print(apriori)
print(tmp.size())
print(tmp.squeeze(-1).size())
# print(pyro.sample("tst", dist.Categorical(tmp)))

RuntimeError: bool value of Tensor with more than one value is ambiguous

In [15]:
num_attr = 10
num_class = 3

mu = pyro.sample('mu_attr', dist.Normal(torch.zeros([num_class, num_attr]), torch.ones([num_class, num_attr])))
sigma = abs(pyro.sample('sigma_attr', dist.Normal(torch.zeros([num_class, num_attr]), torch.ones([num_class, num_attr]))))


In [ ]:
def model(x_data, y_data):
    
    with pyro.plate("decisions", size=decisions_num) as _:
        
        pyro.sampe('class', dist.Categorical(), obs=y_data)

In [ ]:

for i in range(num_clas)
    with pyro.plate("tst", size=num_attr) as _:
        print(pyro.sampe('class', dist.Normal()))

In [ ]:
lista[lista=!lista]
f"std--{clazz}"

In [93]:
ten_a = torch.ones([3, 5])
ten_b = torch.ones([3, 5])
print(ten_a.size())
print(ten_b)
ten_stack = torch.stack([ten_a, ten_b], dim=0)
print(ten_stack)

torch.Size([3, 5])
tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])
tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])
